#### Annotation comparison with human annotators

In [9]:
import pandas as pd
from functools import reduce

In [23]:
tom = pd.read_csv("tom.csv")
thomas = pd.read_csv("thomas.csv")
gaz = pd.read_csv("gareths_annotation.csv")
truth = pd.read_csv("full_sample.csv")

In [ ]:
data_frames = [tom, thomas, gaz, truth]


In [24]:
tom["gender"]

0         male
1         male
2      unknown
3         male
4      unknown
        ...   
95        male
96        male
97        male
98     unknown
99        male
Name: gender, Length: 100, dtype: object

In [ ]:
truth["gender_tom"] = tom["gender"]
truth["gender_gareth"] = 